# Data Cleaning Immo features

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from   cleaning_functions import *

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import unidecode
import math
import pickle

<a id='table'></a>
## Table of Contents
<ul>
<li><a href="#gathering">1. Gathering </a></li>
<li><a href="#assessing">2. Assessing & Cleaning </a></li>
        <ul>
        <li><a href="#buildings">A. Building with apartements </a></li>
            <ul>
                <li><a href="#b_col_selection">i. Preselection of columns </a></li>
                <li><a href="#b_dtypes">ii. Looking at the datatypes, missing values and outliers </a></li>
            </ul>
        <li><a href="#apartments">B. Apartments </a></li>
            <ul>
                <li><a href="#a_col_selection">i. Preselection of columns </a></li>
            </ul>
    </ul>
<li><a href="#cleaning">3. Cleaning </a></li>
</ul>

<a id='gathering'></a>
## 1. Gathering
<a href="#table">Back to the top </a>

**Buildings**

In [2]:
missing_regions = ['anvers', 'limbourg', 'flandre_occidentale', 'flandre_orientale', 'hainaut', 'bruxelles',\
             'luxembourg']

In [3]:
bld_dict = {}

In [4]:
##Loading the different scraped dataframes
with open('./Saved_Variables/20201120_sales_all_regions_immeuble_de_rapport_features.pkl','rb') as f:
    bld_dict["apt_building_part1"] = pickle.load(f)

In [5]:
for region in missing_regions:
    with open(f"./Saved_Variables/20201119_sales_{region}_immeuble_de_rapport_features.pkl",'rb') as f:
        bld_dict[f"apt_building_{region}"] = pickle.load(f)

In [6]:
## Prepa concatenating
# Append all the values of the df dictionnary to a list
df_frames = []
for val in bld_dict.values():
    df_frames.append(val)

In [7]:
 # concatenate the different frames into 1 general df
apt_building_df = pd.concat(df_frames,ignore_index=True)

**Flats**

In [8]:
with open('./Saved_Variables/20201120_sales_all_regions_appartement_features.pkl','rb') as f:
    apartments_df = pickle.load(f)

<a id='assessing'></a>
## 2. Assessing & Cleaning
<a href="#table">Back to the top </a>

In [9]:
apt_building_df.shape

(5015, 133)

In [10]:
for name, df in bld_dict.items():
    print(name)
    print(df.shape)    

apt_building_part1
(1434, 126)
apt_building_anvers
(644, 116)
apt_building_limbourg
(257, 115)
apt_building_flandre_occidentale
(456, 117)
apt_building_flandre_orientale
(630, 118)
apt_building_hainaut
(821, 120)
apt_building_bruxelles
(603, 122)
apt_building_luxembourg
(170, 111)


In [11]:
print(apartments_df.shape)

(7661, 116)


In [12]:
# Let's change the max amount of columns displayed
pd.options.display.max_columns # current setting
pd.set_option("display.max_columns", 128) # changing to 128 for assessing part

In [13]:
pd.options.display.max_rows
pd.set_option("display.max_rows",138)

####  We'll work on a copy of the different dataframes

In [14]:
# We'll work on Copies
buildings = apt_building_df.copy()
apartments = apartments_df.copy()

In [15]:
check_cols_bld = ['region','type_de_zone_inondable','salles_de_bains', 'double_vitrage', 'chambres',
                  'revenu_cadastral', 'type_de_chauffage','surface_du_terrain', 'surface_habitable',
                         'etat_du_batiment', 'annee_de_construction', 'facades', 'classe_energetique']

In [16]:
for feature in check_cols_bld:
    print(feature.upper())
    print("_________________________________")
    print(buildings[feature].value_counts(dropna=False))
    print("=================================")
    print("\n")

REGION
_________________________________
hainaut                821
liege                  655
anvers                 644
flandre-orientale      630
bruxelles              603
flandre-occidentale    456
brabant-flamand        354
namur                  271
limbourg               257
luxembourg             170
brabant-wallon         154
Name: region, dtype: int64


TYPE_DE_ZONE_INONDABLE
_________________________________
NaN                                                                                                                                3090
Zone non inondable                                                                                                                 1815
Zone inondable potentielle                                                                                                           72
Zone d'inondation reconnue                                                                                                           30
Bien immobilier situé tout ou en

In [17]:
check_cols_apt = ['region', 'type_de_zone_inondable','salles_de_bains', 'double_vitrage', 'chambres','type_de_chauffage',
                  'etat_du_batiment','facades','classe_energetique', 'terrasse', 'salles_de_douche',
                         'toilettes', 'parkings_exterieurs','parkings_interieurs', 'cave', 'ascenseur',
                         'nombre_d_etages', 'etage']

In [18]:
for feature in check_cols_apt:
    print(feature.upper())
    print("_________________________________")
    print(apartments[feature].value_counts(dropna=False))
    print("=================================")
    print("\n")

REGION
_________________________________
brabant-flamand        1908
liege                  1517
limbourg               1331
hainaut                1280
brabant-wallon          632
namur                   474
luxembourg              372
bruxelles                48
anvers                   36
flandre-orientale        33
flandre-occidentale      30
Name: region, dtype: int64


TYPE_DE_ZONE_INONDABLE
_________________________________
NaN                                                                                                                                5075
Zone non inondable                                                                                                                 2458
Zone inondable potentielle                                                                                                           89
Zone d'inondation reconnue                                                                                                           28
Bien immobilier situé

#### Splitting in Train and Test Set 

> As we want to keep the _Test Set_ untouched, one of the first things that we do is splitting the dataframe in Train and Test Set

In [19]:
building_train, building_test = train_test_split(buildings, test_size=0.3, random_state=123)
apartment_train, apartment_test = train_test_split(apartments, test_size=0.3, random_state=123)

In [20]:
print(building_train.shape, building_test.shape, apartment_train.shape, apartment_test.shape)

(3510, 133) (1505, 133) (5362, 116) (2299, 116)


<a id='buildings'></a>
### A. Building with apartments
<a href="#table">Back to the top </a>

<a id='b_col_selection'></a>
#### i. Preselection of columns

In [21]:
# Apt Buildings
building_train.sample(5, random_state=123)

,page_url,postcode,city,type_of_good,price,region,immoweb_code,street_property,street_number_property,description,picture_url,disponibilite,disponible_le,quartier_ou_lieu-dit,annee_de_construction,etat_du_batiment,largeur_de_facade,facades,surface_habitable,surface_du_salon,type_de_cuisine,surface_de_la_cuisine,chambres,surface_de_la_chambre_1,surface_de_la_chambre_2,surface_de_la_chambre_3,salles_de_bains,salles_de_douche,toilettes,surface_du_terrain,raccordement_a_l_egout,acces_handicape,acces_securise_/_alarme,consommation_d_energie_primaire,classe_energetique,numero_du_rapport_peb,emission_co2,consommation_theorique_totale_d_energie_primaire,type_de_chauffage,double_vitrage,surface_constructible_totale_au_sol,type_de_zone_inondable,affectation_urbanistique_la_plus_recente_sur_la_base_des_denominations_utilisees_dans_le_registre_des_plans,prix,revenu_cadastral,immeuble_de_rapport,adresse,site_internet,reference_externe,nombre_d_etages,plan_as-built,parkings_interieurs,surface_de_la_chambre_4,surface_de_la_chambre_5,eau_gaz_electricite,parkings_exterieurs,salle_a_manger,surface_de_profession_liberale,cave,grenier,surface_du_jardin,surface_de_la_terrasse,permis_de_batir_obtenu,surface_de_bureau,...,orientation_de_la_terrasse,attestation_de_conformite_des_citernes_a_mazout,salon,porte_blindee,feu_ouvert,dressing,buanderie,profession_liberale,terrain_plat,nombre_d_annexes,pompe_a_chaleur,panneaux_photovoltaiques,visite_virtuelle,informations_supplementaires,pourcentage_loue,orientation_du_jardin,seance_unique,modalites_de_visite,lieu_de_la_vente,mise_a_prix,nom_de_l_agent,etage,terrain_a_front_de_rue,type_de_construction,meuble,ascenseur,revenu_mensuel_actuel,panneaux_solaires,niveau_e_(performance_energetique_globale),jardin,nom_du_bien,chauffage_et/ou_eau_collectifs,jacuzzi,terrain_de_fond,procedure_pour_violation_de_la_reglementation_urbanistique,portes_d_acces,portes_sectionnelles,surface_du_showroom,interphone_video,quais_de_chargement,prix_au_m2,description_libre_peb,sauna,piscine,climatisation,concierge,largeur_de_la_vitrine,autorisation_de_lotissement,hauteur_minimale_disponible,hauteur_maximale_disponible,baies_couvertes,fond_de_commerce_inclus,reception,droit_d_achat_prioritaire_possible,isole,terrain_boise,quais_de_chargement_avec_dispositif_de_levage,portes_coulissantes,,obligation_de_construire,bouquet,rente_mensuelle,nombre_de_rentiers,age_du_rentier
2871,https://www.immoweb.be/fr/annonce/maison/a-ven...,9700,Audenarde,immeuble-de-rapport,670000,flandre-orientale,8978791,Ohiostraat,"203,",,https://static.immoweb.be/photos/0/8/9/7/8/7/9...,NaN,NaN,NaN,1968,Bon,NaN,NaN,266m²mètres carrés,NaN,NaN,NaN,6,NaN,NaN,NaN,3,NaN,3,563m²mètres carrés,Connecté,NaN,NaN,242kWh/m²kilowattheure par mètres carrés,C,20200930-0002321635-RES-2,Non communiqué,Non communiqué,Gaz,Oui,NaN,Zone non inondable,"Zone d'habitat (résidentiel, urbain ou rural)",670000 €,1.826 €1826 €,Oui,Kortrijksesteenweg 749000\n ...,http://www.rosseel.be,GDVC-5118W,NaN,NaN,2,NaN,NaN,Oui,NaN,NaN,NaN,Oui,NaN,NaN,NaN,Oui,NaN,...,NaN,NaN,Oui,Non,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Visite virtuelle,NaN,NaN,Sud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2200,NaN,NaN,Oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
764,https://www.immoweb.be/fr/annonce/immeuble-mix...,5000,Namur,immeuble-de-rapport,795000,namur,8971097,,,Nam Property vous propose ensemble immobilier ...,https://static.immoweb.be/photos/0/8/9/7/1/0/9...,À l'échange des actes,NaN,NaN,1939,À rénover,NaN,2,550m²mètres carrés,NaN,NaN,NaN,5,NaN,NaN,NaN,4,NaN,NaN,180m²mètres carrés,Non connecté,NaN,NaN,Non communiqué,Non communiqué,Non communiqué,Non communiqué,Non communiqué,NaN,NaN,NaN,NaN,NaN,795000 €,NaN,Oui,Rue Emile Cuvelier 85000\n ...,http://www.namproperty.be,4160054,NaN,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [22]:
# Columns names
building_train_cols = list(building_train.columns)
print(building_train_cols)

['page_url', 'postcode', 'city', 'type_of_good', 'price', 'region', 'immoweb_code', 'street_property', 'street_number_property', 'description', 'picture_url', 'disponibilite', 'disponible_le', 'quartier_ou_lieu-dit', 'annee_de_construction', 'etat_du_batiment', 'largeur_de_facade', 'facades', 'surface_habitable', 'surface_du_salon', 'type_de_cuisine', 'surface_de_la_cuisine', 'chambres', 'surface_de_la_chambre_1', 'surface_de_la_chambre_2', 'surface_de_la_chambre_3', 'salles_de_bains', 'salles_de_douche', 'toilettes', 'surface_du_terrain', 'raccordement_a_l_egout', 'acces_handicape', 'acces_securise_/_alarme', 'consommation_d_energie_primaire', 'classe_energetique', 'numero_du_rapport_peb', 'emission_co2', 'consommation_theorique_totale_d_energie_primaire', 'type_de_chauffage', 'double_vitrage', 'surface_constructible_totale_au_sol', 'type_de_zone_inondable', 'affectation_urbanistique_la_plus_recente_sur_la_base_des_denominations_utilisees_dans_le_registre_des_plans', 'prix', 'revenu_c

In [23]:
pct_na_building_train = round((building_train.isnull().sum() / building_train.shape[0])*100, 2).sort_values(ascending=False)
print(pct_na_building_train)

quais_de_chargement_avec_dispositif_de_levage                                                                  100.00
age_du_rentier                                                                                                  99.97
rente_mensuelle                                                                                                 99.97
bouquet                                                                                                         99.97
portes_coulissantes                                                                                             99.97
nombre_de_rentiers                                                                                              99.97
quais_de_chargement                                                                                             99.97
baies_couvertes                                                                                                 99.97
obligation_de_construire                                

In [24]:
bld_apts_features = ['page_url','description','picture_url', 'region','postcode', 'city', 'street_property', 
                     'street_number_property','emission_co2', 'classe_energetique', 'facades', 
                     'etat_du_batiment','surface_habitable', 'surface_du_terrain', 'type_de_chauffage', 
                     'revenu_cadastral','chambres', 'double_vitrage', 'salles_de_bains',
                     'toilettes', 'type_de_cuisine', 'type_de_zone_inondable','eau_gaz_electricite']

In [25]:
round(100*building_train[bld_apts_features].isna().sum()/building_train[bld_apts_features]\
     .shape[0],1).sort_values(ascending=False)

type_de_zone_inondable    61.5
eau_gaz_electricite       60.0
type_de_cuisine           50.9
toilettes                 48.5
salles_de_bains           46.6
revenu_cadastral          45.0
double_vitrage            44.7
chambres                  43.2
type_de_chauffage         39.9
surface_du_terrain        33.4
etat_du_batiment          30.0
surface_habitable         28.8
facades                   26.4
classe_energetique         1.1
emission_co2               1.1
street_number_property     0.0
street_property            0.0
city                       0.0
postcode                   0.0
region                     0.0
picture_url                0.0
description                0.0
page_url                   0.0
dtype: float64

In [26]:
building_train[bld_apts_features].sample(5, random_state=123)

,page_url,description,picture_url,region,postcode,city,street_property,street_number_property,emission_co2,classe_energetique,facades,etat_du_batiment,surface_habitable,surface_du_terrain,type_de_chauffage,revenu_cadastral,chambres,double_vitrage,salles_de_bains,toilettes,type_de_cuisine,type_de_zone_inondable,eau_gaz_electricite
2871,https://www.immoweb.be/fr/annonce/maison/a-ven...,,https://static.immoweb.be/photos/0/8/9/7/8/7/9...,flandre-orientale,9700,Audenarde,Ohiostraat,"203,",Non communiqué,C,NaN,Bon,266m²mètres carrés,563m²mètres carrés,Gaz,1.826 €1826 €,6,Oui,3,3,NaN,Zone non inondable,Oui
764,https://www.immoweb.be/fr/annonce/immeuble-mix...,Nam Property vous propose ensemble immobilier ...,https://static.immoweb.be/photos/0/8/9/7/1/0/9...,namur,5000,Namur,,,Non communiqué,Non communiqué,2,À rénover,550m²mètres carrés,180m²mètres carrés,NaN,NaN,5,NaN,4,NaN,NaN,NaN,NaN
2768,https://www.immoweb.be/fr/annonce/immeuble-a-a...,,https://static.immoweb.be/photos/0/8/2/6/0/2/8...,flandre-occidentale,8400,Ostende,,,Non communiqué,C,2,NaN,NaN,70m²mètres carrés,NaN,1.845 €1845 €,9,NaN,NaN,4,NaN,NaN,NaN
2870,https://www.immoweb.be/fr/annonce/maison/a-ven...,,https://static.immoweb.be/photos/0/8/9/5/7/6/5...,flandre-orientale,9900,Eeklo,Pokmoere,"33,",Non communiqué,E,NaN,À rénover,280m²mètres carrés,589m²mètres carrés,Gaz,1.383 €1383 €,6,Oui,2,2,NaN,Zone non inondable,Oui
2285,https://www.immoweb.be/fr/annonce/immeuble-ind...,,https://static.immoweb.be/photos/0/8/8/8/0/3/4...,limbourg,3500,Hasselt,Herkenrodesingel,10,Non communiqué,Non communiqué,NaN,Excellent état,221m²mètres carrés,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<a id='b_dtypes'></a>
#### ii. Looking at the datatypes, missing values and outliers
<a href="#table">Back to the top </a>

In [27]:
selected_bld_cols = [ 'type_de_zone_inondable', 'salles_de_bains', 'double_vitrage', 'chambres', 'revenu_cadastral', 'type_de_chauffage', 
                'surface_du_terrain', 'surface_habitable', 'etat_du_batiment', 'annee_de_construction',
                'facades', 'classe_energetique']

In [28]:
for col in selected_bld_cols:
    print(col.upper() + " : ", "\n")
    print(building_train[col].value_counts(dropna=False) ,"\n")

TYPE_DE_ZONE_INONDABLE :  

NaN                                                                                                                                2159
Zone non inondable                                                                                                                 1284
Zone inondable potentielle                                                                                                           42
Zone d'inondation reconnue                                                                                                           20
Bien immobilier situé tout ou en partie dans une zone inondable délimitée                                                             2
Bien immobilier situé tout ou en partie dans une zone riveraine délimitée                                                             1
Zone riveraine circonscrite possible                                                                                                  1
Bien immobilier situ

**Any duplicates?**

In [29]:
building_train.duplicated(subset=['immoweb_code'], keep='first').sum()

0

**Buildings where the street and number weren't given**

In [30]:
building_train[building_train['street_property']==""].shape

(1098, 133)

In [31]:
building_train[building_train['street_property']==""].sample(5,random_state=123)

,page_url,postcode,city,type_of_good,price,region,immoweb_code,street_property,street_number_property,description,picture_url,disponibilite,disponible_le,quartier_ou_lieu-dit,annee_de_construction,etat_du_batiment,largeur_de_facade,facades,surface_habitable,surface_du_salon,type_de_cuisine,surface_de_la_cuisine,chambres,surface_de_la_chambre_1,surface_de_la_chambre_2,surface_de_la_chambre_3,salles_de_bains,salles_de_douche,toilettes,surface_du_terrain,raccordement_a_l_egout,acces_handicape,acces_securise_/_alarme,consommation_d_energie_primaire,classe_energetique,numero_du_rapport_peb,emission_co2,consommation_theorique_totale_d_energie_primaire,type_de_chauffage,double_vitrage,surface_constructible_totale_au_sol,type_de_zone_inondable,affectation_urbanistique_la_plus_recente_sur_la_base_des_denominations_utilisees_dans_le_registre_des_plans,prix,revenu_cadastral,immeuble_de_rapport,adresse,site_internet,reference_externe,nombre_d_etages,plan_as-built,parkings_interieurs,surface_de_la_chambre_4,surface_de_la_chambre_5,eau_gaz_electricite,parkings_exterieurs,salle_a_manger,surface_de_profession_liberale,cave,grenier,surface_du_jardin,surface_de_la_terrasse,permis_de_batir_obtenu,surface_de_bureau,...,orientation_de_la_terrasse,attestation_de_conformite_des_citernes_a_mazout,salon,porte_blindee,feu_ouvert,dressing,buanderie,profession_liberale,terrain_plat,nombre_d_annexes,pompe_a_chaleur,panneaux_photovoltaiques,visite_virtuelle,informations_supplementaires,pourcentage_loue,orientation_du_jardin,seance_unique,modalites_de_visite,lieu_de_la_vente,mise_a_prix,nom_de_l_agent,etage,terrain_a_front_de_rue,type_de_construction,meuble,ascenseur,revenu_mensuel_actuel,panneaux_solaires,niveau_e_(performance_energetique_globale),jardin,nom_du_bien,chauffage_et/ou_eau_collectifs,jacuzzi,terrain_de_fond,procedure_pour_violation_de_la_reglementation_urbanistique,portes_d_acces,portes_sectionnelles,surface_du_showroom,interphone_video,quais_de_chargement,prix_au_m2,description_libre_peb,sauna,piscine,climatisation,concierge,largeur_de_la_vitrine,autorisation_de_lotissement,hauteur_minimale_disponible,hauteur_maximale_disponible,baies_couvertes,fond_de_commerce_inclus,reception,droit_d_achat_prioritaire_possible,isole,terrain_boise,quais_de_chargement_avec_dispositif_de_levage,portes_coulissantes,,obligation_de_construire,bouquet,rente_mensuelle,nombre_de_rentiers,age_du_rentier
4789,https://www.immoweb.be/fr/annonce/immeuble-a-a...,1190,Forest,immeuble-de-rapport,615000,bruxelles,8711496,,,A proximité immédiate de la Place de l'Altitud...,https://static.immoweb.be/photos/0/8/7/1/1/4/9...,À l'échange des actes,NaN,NaN,NaN,NaN,6 m,2,300m²mètres carrés,NaN,NaN,NaN,4,NaN,NaN,NaN,4,NaN,NaN,NaN,Connecté,NaN,NaN,484kWh/m²kilowattheure par mètres carrés,G,Non communiqué,Non communiqué,Non communiqué,Gaz,Oui,NaN,NaN,NaN,615000 €,NaN,Oui,Chaussée d Alsemberg 2561190\n ...,NaN,4025187,NaN,NaN,NaN,NaN,NaN,Oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3754,https://www.immoweb.be/fr/annonce/immeuble-a-a...,6061,Montignies-sur-Sambre,immeuble-de-rapport,175000,hainaut,8980247,,,Montignies-sur-Sambre: 175.000€ : Immeuble de ...,https://static.immoweb.be/photos/0/8/9/8/0/2/4...,NaN,NaN,NaN,NaN,Bon,NaN,3,220m²mètres carrés,NaN,Équipée,NaN,3,NaN,NaN,NaN,1,1,3,NaN,NaN,NaN,NaN,582kWh/m²kilowattheure par mètres carrés,G,20171114005053,Non communiqué,54196 kWh/an,Gaz,Oui,NaN,NaN,NaN,175000 €,NaN,Oui,Boulevard Audent 176000\n ...,http://www.lapropriete.be,11710 - 3090,NaN,Non,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oui,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

#### Some Cleaning Comments:

##### Feature: eau_gaz_electricite

> As there're more than 50% of missing values and all the rest is "Oui" (="yes), we'll start by dropping this feature

##### Feature: type_de_zone_inondable

> - We'll make the asumption that a missing value for "type_de_zone_inondable" (= type of flood zone), means that the building isn't in a potential or recognized flood zone.
> - Beside this, we'll keep only 2 categories (to lower the total amount of features. That means that we'll mark all NaN as "No_flood_zone" and the rest as "Potential_flood_zone". We'll then encode label encode them as 0 and 1 respectively.

##### Feature: type_de_cuisine

In [32]:
building_train[building_train.type_de_cuisine.isna()][['region','immoweb_code','city']]\
.sample(5, random_state=123)

,region,immoweb_code,city
4068,hainaut,8940577,Charleroi
2157,limbourg,9000133,Hasselt
1151,brabant-flamand,8003338,Diest
567,liege,8904452,Eupen
716,namur,8442405,AUVELAIS


> - https://www.immoweb.be/fr/annonce/maison/a-vendre/liege/4000/8956821 : house with works so at this point doesn't have a kitchen
- https://www.immoweb.be/fr/annonce/maison/a-vendre/sambreville/5060/8684334 : not clear if there's a kitchen (probably well)
- https://www.immoweb.be/fr/annonce/maison/a-vendre/virton/6760/8948635: there's a kitchen
- https://www.immoweb.be/fr/annonce/immeuble-industriel-mixte/a-vendre/houdeng-goegnies/7110/8614276?searchId=5f6d01f488593: no kitchen but here those are industrial buildings
- https://www.immoweb.be/fr/annonce/immeuble-a-appartements/a-vendre/bouillon/6830/8814576?searchId=5f8738e9d2712: there are kitchens

***Comments :***
> - At this stage, as biggest part is NaN, and a small percentage is hyper-equiped, let's start the modelling without this features

##### Feature: 'toilettes' and 'salles_de_bains"

In [33]:
sdb_Xb = building_train[building_train['salles_de_bains'].notnull()][['region','immoweb_code','city', 'postcode','prix',\
                                                          'toilettes', 'salles_de_bains']]

In [34]:
sdb_Xb['salles_de_bains']  = sdb_Xb['salles_de_bains'].astype('int')

In [35]:
sdb_Xb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1875 entries, 3660 to 1593
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   region           1875 non-null   object
 1   immoweb_code     1875 non-null   object
 2   city             1875 non-null   object
 3   postcode         1875 non-null   object
 4   prix             1860 non-null   object
 5   toilettes        1344 non-null   object
 6   salles_de_bains  1875 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 117.2+ KB


In [36]:
sdb_Xb[sdb_Xb['salles_de_bains']>9].sort_values(by="salles_de_bains", ascending=False)

,region,immoweb_code,city,postcode,prix,toilettes,salles_de_bains
209,liege,8611427,Verviers,4800,2450000 €,38,35
4261,bruxelles,7701433,Bruxelles,1000,5200000 €,30,29
2591,flandre-occidentale,8648099,Oostende,8400,3500000 €,29,29
4458,bruxelles,9030017,Saint-Gilles,1060,1650000 €,NaN,21
73,liege,8888668,Liege,4020,630000 €,18,16
581,liege,8972116,Trois-ponts,4980,875000 €,16,16
2950,flandre-orientale,9004657,Wetteren,9230,2497000 €,16,16
744,namur,8725629,Gedinne,5575,1499000 €,NaN,15
2528,flandre-occidentale,8668018,Heist-aan-Zee,8301,2400000 €,15,15
1636,anvers,8855356,Anvers,2060,1349000 €,4,15


**Comments :**
> - 8611427: immoweb.be/fr/annonce/maison/a-vendre/verviers/4800/8611427 => has indeed 35 bathrooms, as there are 35 units
- 8668018: https://www.immoweb.be/en/classified/house/for-sale/heist-aan-zee/8301/8668018 => has 15 bathroom
- 7953060 : https://www.immoweb.be/fr/annonce/immeuble-commercial-mixte/a-vendre/la-roche-en-ardenne/6980/7953060?searchId=5f3de12b0e1c5 => has 10 bathrooms


> - We won't keep the amount of toilettes for the moment 
- Let's fill the NaN values with the most_frequent value

##### Feature: 'double_vitrage'

In [37]:
building_train[building_train['double_vitrage'].isna()][['region','immoweb_code','city','double_vitrage',\
                                                         'prix','classe_energetique','emission_co2']]\
                                                                                    .sample(5,random_state=123)

,region,immoweb_code,city,double_vitrage,prix,classe_energetique,emission_co2
4723,bruxelles,8734217,Evere,NaN,860000 €,G,Non communiqué
2920,flandre-orientale,8950249,Erpe-Mere,NaN,139000 €,F,Non communiqué
2072,anvers,8899904,Anvers,NaN,590000 €,Non communiqué,Non communiqué
4843,bruxelles,7921662,Jette,NaN,3750000 €,Non communiqué,Non communiqué
3849,hainaut,8889126,Lessines,NaN,225000 €,F,Non communiqué


**Comments :**
> - Here as there is only one value: "oui" or np.nan => we'll mark the NaN with mention "not_given"
- Feeling this is related to Energetical class => later we'll probably keep only one of both

##### Feature: 'chambres'

**Comments :**
> - After investigating different goods with NaN for the amount of rooms, we noticed that amongst them there are building of offices, restaurants, but sometimes real houses with supposedly some rooms
- So as it's difficult in those conditions to impute with specific value, we'll replace the NaN's by -1

#####  Feature: 'type_de_chauffage'

In [38]:
list(building_train[building_train['type_de_chauffage'].isna()][ 'picture_url'].sample(5, random_state=123))

['https://static.immoweb.be/photos/0/8/5/5/1/1/5/9/8551159_1.jpg?cache=20200111162256',
 'https://static.immoweb.be/photos/0/9/0/3/8/5/5/0/9038550_1.jpg?cache=20201116214417',
 'https://static.immoweb.be/photos/0/8/4/3/1/8/3/4/8431834_1.jpg?cache=20191029115926',
 'https://static.immoweb.be/photos/0/7/8/2/0/0/7/5/7820075_1.jpg?cache=20181106044416',
 'https://static.immoweb.be/photos/0/8/7/0/3/8/9/8/8703898_1.jpg?cache=20200508035925']

##### Feature: 'surface_du_terrain'

**Comments :**
> -  Devide the 'surface_de_terrain' in quantiles segments

##### Feature: 'classe_energetique'

**Comments :**
> - We'll group A+ and A++ together with A
> - For the energy levels containing an '_', we'll remove it and keep the second energy level which is a bit less worse
> - Replace NA by "not_given"

##### Feature: 'emission_co2'

**Comments :**
> - as for a majority of the buildings the CO2 emission isn't given, we won't use this feature but rely more on the 'classe_energetique'

##### Region, city and/or postcode

**Comments:**
> - At this point, we prefer to keep postcode over city, as in city there can be more easily some typing/webscraping mistakes. And we keep postcode over region because it's more granular.

_**Encoding of postcodes**_

**Idea :**
> - encoding with latitudes and longitudes

 source for info: https://github.com/jief/zipcode-belgium
> - possibility to add extra demographical information


<a id='apartments'></a>
### B. Apartments
<a href="#table">Back to the top </a>

<a id='a_col_selection'></a>
#### i. Preselection of columns

In [39]:
apartments_df.sample(5, random_state=123)

,page_url,postcode,city,type_of_good,price,region,immoweb_code,street_property,street_number_property,description,picture_url,disponibilite,quartier_ou_lieu-dit,annee_de_construction,etage,nombre_d_etages,etat_du_batiment,facades,parkings_interieurs,surface_habitable,surface_du_salon,type_de_cuisine,surface_de_la_cuisine,chambres,surface_de_la_chambre_1,salles_de_bains,toilettes,cave,surface_de_la_terrasse,ascenseur,consommation_d_energie_primaire,classe_energetique,numero_du_rapport_peb,emission_co2,consommation_theorique_totale_d_energie_primaire,plan_as-built,type_de_chauffage,double_vitrage,type_de_zone_inondable,prix,revenu_cadastral,adresse,site_internet,reference_externe,largeur_de_facade,parkings_exterieurs,feu_ouvert,surface_de_la_chambre_2,salles_de_douche,surface_de_la_cave,porte_blindee,parlophone,attestation_de_conformite_des_citernes_a_mazout,permis_de_batir_obtenu,nom_du_bien,visite_virtuelle,niveau_e_(performance_energetique_globale),dressing,meuble,terrasse,interphone_video,surface_de_la_chambre_3,droit_d_achat_prioritaire_possible,buanderie,concierge,acces_handicape,chauffage_et/ou_eau_collectifs,autorisation_de_lotissement,piscine,immeuble_de_rapport,salle_a_manger,jardin,orientation_du_jardin,salon,bureau,surface_du_jardin,disponible_le,surface_du_grenier,orientation_de_la_terrasse,procedure_pour_violation_de_la_reglementation_urbanistique,seance_unique,modalites_de_visite,lieu_de_la_vente,mise_a_prix,nom_de_l_agent,pompe_a_chaleur,grenier,acces_securise_/_alarme,surface_constructible_totale_au_sol,panneaux_solaires,panneaux_photovoltaiques,surface_de_bureau,informations_supplementaires,type_de_construction,nombre_d_annexes,revenu_mensuel_actuel,isole,climatisation,surface_de_la_chambre_4,surface_de_profession_liberale,surface_de_la_chambre_5,valeur_du_bien,bouquet,rente_mensuelle,rente_indexee,vente_en_nue_propriete,duree_maximale_de_la_rente,obligation_de_construire,rente_reversible,nombre_de_rentiers,age_des_rentiers,profession_liberale,age_du_rentier,description_libre_peb,jacuzzi,sauna
3586,https://www.immoweb.be/fr/annonce/appartement/...,4000,Liège,appartement,182500,liege,8367880,Rue Renardi,169,"À 100 m du golf de Bernalmont, cet appartement...",https://static.immoweb.be/photos/0/8/3/6/7/8/8...,NaN,NaN,2015,2,3,Excellent état,3,1,122m²mètres carrés,40m²mètres carrés,Équipée,NaN,3,NaN,1,1,Oui,NaN,NaN,113kWh/m²kilowattheure par mètres carrés,B,20140327041703,Non communiqué,13522 kWh/an,Non,Gaz,Oui,NaN,182500 €,965 €965 €,"rue Louvrex, 984000\n ...",http://www.optimum.be,7725 - 17/10/09,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1396,https://www.immoweb.be/fr/annonce/appartement/...,8400,Ostende,appartement,229000,flandre-occidentale,6472703,Hendrik Serruyslaan,60/6L,"Bel appartement avec 2 chambres à coucher, ter...",https://static.immoweb.be/photos/0/6/4/7/2/7/0...,NaN,NaN,1950,6,NaN,Bon,2,NaN,80m²mètres carrés,26m²mètres carrés,Équipée,8m²mètres carrés,2,11m²mètres carrés,1,1,NaN,3m²mètres carrés,Oui,591kWh/m²kilowattheure par mètres carrés,F,1412703,Non communiqué,Non communiqué,Non,Gaz,Oui,Zone non inondable,229000 €,1.073 €1073 €,Karel Janssenslaan 18400\n ...,http://www.dermul.be,spir100266 RUYTINGEN,NaN,NaN,NaN,13m²mètres carrés,NaN,NaN,Non,Oui,Non,Oui,NaN,NaN,NaN,NaN,NaN,NaN,Oui,NaN,Oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6353,https://www.immoweb.be/fr/annonce/appartement/...,1650,Beersel,appartement,250000,brabant-flamand,8978327,Zonienwoudlaan,7,IDÉAL POUR INVESTISSEUR - RENDEMENT 5% - Spaci...,https://static.immoweb.be/photos/0/8/9/7/8/3/2...,À l'échange des actes,Sept Fontaines,

In [40]:
apartments_cols = list(apartments_df.columns)
print(apartments_cols)

['page_url', 'postcode', 'city', 'type_of_good', 'price', 'region', 'immoweb_code', 'street_property', 'street_number_property', 'description', 'picture_url', 'disponibilite', 'quartier_ou_lieu-dit', 'annee_de_construction', 'etage', 'nombre_d_etages', 'etat_du_batiment', 'facades', 'parkings_interieurs', 'surface_habitable', 'surface_du_salon', 'type_de_cuisine', 'surface_de_la_cuisine', 'chambres', 'surface_de_la_chambre_1', 'salles_de_bains', 'toilettes', 'cave', 'surface_de_la_terrasse', 'ascenseur', 'consommation_d_energie_primaire', 'classe_energetique', 'numero_du_rapport_peb', 'emission_co2', 'consommation_theorique_totale_d_energie_primaire', 'plan_as-built', 'type_de_chauffage', 'double_vitrage', 'type_de_zone_inondable', 'prix', 'revenu_cadastral', 'adresse', 'site_internet', 'reference_externe', 'largeur_de_facade', 'parkings_exterieurs', 'feu_ouvert', 'surface_de_la_chambre_2', 'salles_de_douche', 'surface_de_la_cave', 'porte_blindee', 'parlophone', 'attestation_de_conform

In [41]:
# Percentage of missing values
pct_na_apts = round((apartments_df.isnull().sum() / apartments_df.shape[0])*100, 1).sort_values(ascending=False)
print(pct_na_apts)

sauna                                                         100.0
jacuzzi                                                       100.0
revenu_mensuel_actuel                                         100.0
description_libre_peb                                         100.0
age_du_rentier                                                 99.9
age_des_rentiers                                               99.9
rente_reversible                                               99.9
obligation_de_construire                                       99.9
surface_de_profession_liberale                                 99.9
profession_liberale                                            99.9
valeur_du_bien                                                 99.8
isole                                                          99.8
procedure_pour_violation_de_la_reglementation_urbanistique     99.8
surface_de_la_chambre_5                                        99.8
rente_mensuelle                                 

In [42]:
apt_features = ['price', 'description', 'picture_url', 'region', 'postcode', 'city', 'street_property', 
                'street_number_property', 'classe_energetique', 'etat_du_batiment','surface_habitable',
                'type_de_chauffage', 'revenu_cadastral','chambres', 'double_vitrage', 'salles_de_bains',
                'toilettes', 'type_de_cuisine', 'type_de_zone_inondable', 'orientation_de_la_terrasse', 
                'parkings_interieurs', 'cave', 'annee_de_construction', 'etage','ascenseur', 
                'surface_de_la_cuisine', 'terrasse', 'nombre_d_etages', 'salles_de_douche', 
                'parkings_exterieurs']

In [43]:
# Percentage of missing values
apt_df = apartments_df[apt_features]

pct_na_apts = round((apt_df.isna().sum()/apt_df.shape[0])*100,1).sort_values(ascending=False)

print(pct_na_apts)

orientation_de_la_terrasse    92.4
terrasse                      85.8
salles_de_douche              79.2
parkings_interieurs           74.6
parkings_exterieurs           74.4
surface_de_la_cuisine         73.3
cave                          70.4
revenu_cadastral              66.9
type_de_zone_inondable        66.2
ascenseur                     51.9
nombre_d_etages               50.9
etage                         44.6
annee_de_construction         42.5
type_de_cuisine               34.7
etat_du_batiment              34.0
type_de_chauffage             34.0
double_vitrage                32.6
toilettes                     27.9
salles_de_bains               19.3
surface_habitable             10.3
chambres                       3.1
classe_energetique             0.1
street_number_property         0.0
street_property                0.0
city                           0.0
postcode                       0.0
region                         0.0
picture_url                    0.0
description         

In [44]:
print(list(pct_na_apts.index))

['orientation_de_la_terrasse', 'terrasse', 'salles_de_douche', 'parkings_interieurs', 'parkings_exterieurs', 'surface_de_la_cuisine', 'cave', 'revenu_cadastral', 'type_de_zone_inondable', 'ascenseur', 'nombre_d_etages', 'etage', 'annee_de_construction', 'type_de_cuisine', 'etat_du_batiment', 'type_de_chauffage', 'double_vitrage', 'toilettes', 'salles_de_bains', 'surface_habitable', 'chambres', 'classe_energetique', 'street_number_property', 'street_property', 'city', 'postcode', 'region', 'picture_url', 'description', 'price']


In [45]:
selected_cols = ['type_de_zone_inondable','salles_de_bains', 'double_vitrage', 'chambres', 'revenu_cadastral',
                'surface_habitable','etat_du_batiment', 'annee_de_construction','facades', 
                'classe_energetique', 'terrasse', 'salles_de_douche','toilettes','parkings_exterieurs',
                'parkings_interieurs', 'cave', 'ascenseur', 'nombre_d_etages',  'etage']

In [46]:
for col in selected_cols:
    print(col.upper() + " : ", "\n")
    print(apartments[col].value_counts(dropna=False) ,"\n")


TYPE_DE_ZONE_INONDABLE :  

NaN                                                                                                                                5075
Zone non inondable                                                                                                                 2458
Zone inondable potentielle                                                                                                           89
Zone d'inondation reconnue                                                                                                           28
Bien immobilier situé tout ou en partie dans une zone inondable délimitée                                                             7
Bien immobilier situé tout ou en partie dans une zone riveraine délimitée                                                             2
Bien immobilier situé entièrement ou partiellement dans une zone potentiellement inondable et dans une zone riveraine délimitée       1
Zone riveraine circo

<a id='cleaning'></a>
## 3. Cleaning
<a href="#table">Back to the top </a>

### Buildings

**Train Set**

In [47]:
bld_imp_most_freq, bld_imp_median, bld_qtl_cadastre, bld_qtl_terrain, building_train_new = cleaning_df(
                                                                                            building_train)

In [48]:
building_train_encoded = encoding_df(building_train_new)

**Test Set**

In [49]:
_, _, _, _, building_test_new = cleaning_df(building_test, df_type='test', na_imputer_most_freq=bld_imp_most_freq,
                                     na_imputer_median=bld_imp_median, qtl_cadastre=bld_qtl_cadastre,
                                      qtl_terrain=bld_qtl_terrain)

In [50]:
building_test_encoded = encoding_df(building_test_new)

In [51]:
building_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3510 entries, 3660 to 3582
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   immoweb_code                   3510 non-null   object 
 1   price                          3510 non-null   int64  
 2   region                         3510 non-null   object 
 3   postcode                       3510 non-null   object 
 4   street_property                3510 non-null   object 
 5   street_number_property         3510 non-null   object 
 6   description                    3510 non-null   object 
 7   type_de_zone_inondable         3510 non-null   int64  
 8   salles_de_bains                3510 non-null   int64  
 9   double_vitrage                 3510 non-null   int64  
 10  chambres                       3510 non-null   int64  
 11  revenu_cadastral               3510 non-null   int64  
 12  surface_du_terrain             3510 non-null 

In [52]:
building_train_encoded.head()

,immoweb_code,price,region,postcode,street_property,street_number_property,description,type_de_zone_inondable,salles_de_bains,double_vitrage,chambres,revenu_cadastral,surface_du_terrain,surface_habitable,etat_du_batiment,annee_de_construction,facades,classe_energetique,type_de_chauffage_alternative,type_de_chauffage_electrique,type_de_chauffage_gaz,type_de_chauffage_mazout,type_de_chauffage_not_given,long,lat
3660,8742110,215000,hainaut,7301,wasmes,33,REF 5474 - HORNU. Immeuble de rapport de ty...,0,1,1,4,1,2,100,3,1962,3,4,0,0,1,0,0,3.827608,50.433761
1063,8554928,635000,brabant-flamand,3080,,,~Affaire exceptionnelle selon agents~---IMMEUB...,0,1,0,7,-1,2,245,3,1962,3,-1,0,0,0,0,1,4.556566,50.834883
2265,8916926,498000,limbourg,3960,,,,0,2,0,4,4,4,930,1,1993,4,2,0,0,1,0,0,5.678853,51.130552
238,9033657,225000,liege,4000,Rue Saint-Gilles,8,"Situé en plein coeur du centre-ville, à l'entr...",0,1,1,2,2,-1,125,3,1950,2,3,0,0,1,0,0,5.547616,50.678042
4895,9011257,895000,luxembourg,6767,rue de l'Ermitage,18,W Immobilière vous propose dans le charmant vi...,0,10,0,10,4,4,858,3,1962,2,-1,0,0,0,0,1,5.473504,49.508502


In [53]:
building_test_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1505 entries, 1428 to 2287
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   immoweb_code                   1505 non-null   object 
 1   price                          1505 non-null   int64  
 2   region                         1505 non-null   object 
 3   postcode                       1505 non-null   object 
 4   street_property                1505 non-null   object 
 5   street_number_property         1505 non-null   object 
 6   description                    1505 non-null   object 
 7   type_de_zone_inondable         1505 non-null   int64  
 8   salles_de_bains                1505 non-null   int64  
 9   double_vitrage                 1505 non-null   int64  
 10  chambres                       1505 non-null   int64  
 11  revenu_cadastral               1505 non-null   int64  
 12  surface_du_terrain             1505 non-null 

In [54]:
building_test_encoded.head()

,immoweb_code,price,region,postcode,street_property,street_number_property,description,type_de_zone_inondable,salles_de_bains,double_vitrage,chambres,revenu_cadastral,surface_du_terrain,surface_habitable,etat_du_batiment,annee_de_construction,facades,classe_energetique,type_de_chauffage_alternative,type_de_chauffage_electrique,type_de_chauffage_gaz,type_de_chauffage_mazout,type_de_chauffage_not_given,long,lat
1428,9043986,350000,brabant-wallon,1380,,,Villa implantée sur un terrain de 8 ares 53 ce...,0,1,0,2,3,4,145,5,1962,4,-1,0,0,0,0,1,4.421212,50.662548
4969,8004061,180000,luxembourg,6850,Rue Paul Verlaine,12,Bel immeuble 3 façades en pierres du pays comp...,0,1,1,-1,3,3,203,3,1962,3,5,0,0,0,1,0,5.134904,49.903527
77,9028014,249000,liege,4000,Rue Lamarck,137,Une grande maison (idéale pour une profession ...,0,1,1,3,3,3,268,1,1962,2,4,0,0,1,0,0,5.547616,50.678042
4556,8848413,569000,bruxelles,1080,,,MOLENBEEK-SAINT-JEAN - REF : 4097730 - Quartie...,0,7,1,7,2,-1,225,4,1962,2,-1,0,0,1,0,0,4.322778,50.854355
2167,9028423,425000,limbourg,3500,kuringersteenweg,132,,0,1,1,-1,3,2,300,2,1953,3,-1,0,0,1,0,0,5.311946,50.903584


### Apartments

**Train**

In [55]:
_, _, _, _, building_test_new = cleaning_df(building_test, df_type='test', na_imputer_most_freq=bld_imp_most_freq,
                                     na_imputer_median=bld_imp_median, qtl_cadastre=bld_qtl_cadastre,
                                      qtl_terrain=bld_qtl_terrain)

In [56]:
apt_imp_most_freq, apt_imp_median, apt_qtl_cadastre, apt_qtl_terrain, apartment_train_new = \
                                                        cleaning_df(apartment_train, type_of_good='apartment')

In [57]:
apartment_train_encoded = encoding_df(apartment_train_new, type_of_good='apartment')

**Test**

In [58]:
_, _, _, _, apartment_test_new = cleaning_df(apartment_test, type_of_good='apartment', df_type='test',
                                       na_imputer_most_freq=apt_imp_most_freq, 
                                       na_imputer_median=apt_imp_median,
                                       qtl_cadastre=apt_qtl_cadastre, qtl_terrain=apt_qtl_terrain)

In [59]:
apartment_test_encoded = encoding_df(apartment_test_new, type_of_good='apartment')

In [60]:
apartment_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5342 entries, 5555 to 3582
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   immoweb_code                   5342 non-null   object 
 1   price                          5342 non-null   int64  
 2   region                         5342 non-null   object 
 3   postcode                       5342 non-null   object 
 4   street_property                5342 non-null   object 
 5   street_number_property         5342 non-null   object 
 6   description                    5342 non-null   object 
 7   type_de_zone_inondable         5342 non-null   int64  
 8   salles_de_bains                5342 non-null   int64  
 9   double_vitrage                 5342 non-null   int64  
 10  chambres                       5342 non-null   int64  
 11  revenu_cadastral               5342 non-null   int64  
 12  surface_habitable              5342 non-null 

In [61]:
apartment_train_encoded.sample(3, random_state=123)

,immoweb_code,price,region,postcode,street_property,street_number_property,description,type_de_zone_inondable,salles_de_bains,double_vitrage,chambres,revenu_cadastral,surface_habitable,etat_du_batiment,annee_de_construction,facades,classe_energetique,terrasse,salles_de_douche,toilettes,parkings_exterieurs,parkings_interieurs,cave,ascenseur,nombre_d_etages,etage,type_de_chauffage_alternative,type_de_chauffage_electrique,type_de_chauffage_gaz,type_de_chauffage_mazout,type_de_chauffage_not_given,long,lat
1214,7452103,248300,limbourg,3970,Stationsstraat,2.3,,0,1,1,2,-1,104,1,2019,2,-1,0,0,1,0,0,0,1,2,1,0,1,0,0,0,5.263892,51.122058
4647,8633426,365000,namur,5100,Rue de Geronsart,280,"JAMBES, sis dans un écrin de verdure - splendi...",0,2,1,3,1,162,1,2018,2,2,0,1,3,1,0,0,1,2,1,0,0,1,0,0,4.949745,50.428628
4396,8880508,185000,liege,4420,,,"OPTION !!!! Superbe appartement 1 chambre, sit...",0,1,0,1,2,62,1,2016,2,2,0,0,1,1,1,1,0,2,1,0,1,0,0,0,5.530706,50.620651


In [62]:
for col in selected_cols:
    print(col.upper() , '\n')
    print(apartment_train_encoded[col].value_counts(), '\n')

TYPE_DE_ZONE_INONDABLE 

0    5249
1      93
Name: type_de_zone_inondable, dtype: int64 

SALLES_DE_BAINS 

1     4848
2      448
3       34
5        4
4        4
11       1
10       1
12       1
8        1
Name: salles_de_bains, dtype: int64 

DOUBLE_VITRAGE 

1    3585
0    1757
Name: double_vitrage, dtype: int64 

CHAMBRES 

 2      2936
 3      1131
 1       958
-1       167
 4       107
 5        18
 6         7
 8         5
 204       3
 22        2
 10        2
 7         1
 25        1
 9         1
 30        1
 16        1
 27        1
Name: chambres, dtype: int64 

REVENU_CADASTRAL 

-1    3578
 1     444
 2     442
 4     441
 3     437
Name: revenu_cadastral, dtype: int64 

SURFACE_HABITABLE 

95     661
90     153
85     137
100    135
80     134
      ... 
22       1
26       1
202      1
214      1
445      1
Name: surface_habitable, Length: 276, dtype: int64 

ETAT_DU_BATIMENT 

1    3803
3    1081
2     237
4     153
5      68
Name: etat_du_batiment, dtype: int64 

ANN

In [63]:
apartment_test_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2298 entries, 3586 to 5916
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   immoweb_code                   2298 non-null   object 
 1   price                          2298 non-null   int64  
 2   region                         2298 non-null   object 
 3   postcode                       2298 non-null   object 
 4   street_property                2298 non-null   object 
 5   street_number_property         2298 non-null   object 
 6   description                    2298 non-null   object 
 7   type_de_zone_inondable         2298 non-null   int64  
 8   salles_de_bains                2298 non-null   int64  
 9   double_vitrage                 2298 non-null   int64  
 10  chambres                       2298 non-null   int64  
 11  revenu_cadastral               2298 non-null   int64  
 12  surface_habitable              2298 non-null 

In [64]:
apartment_train_encoded.head()

,immoweb_code,price,region,postcode,street_property,street_number_property,description,type_de_zone_inondable,salles_de_bains,double_vitrage,chambres,revenu_cadastral,surface_habitable,etat_du_batiment,annee_de_construction,facades,classe_energetique,terrasse,salles_de_douche,toilettes,parkings_exterieurs,parkings_interieurs,cave,ascenseur,nombre_d_etages,etage,type_de_chauffage_alternative,type_de_chauffage_electrique,type_de_chauffage_gaz,type_de_chauffage_mazout,type_de_chauffage_not_given,long,lat
5555,8484020,153000,brabant-flamand,3300,Zijdelingsestraat,"1,b",,0,1,1,1,-1,75,5,2001,2,2,0,0,1,0,0,1,0,1,2,0,0,0,0,1,4.911143,50.843790
5903,8888259,265000,brabant-flamand,1730,Kalkoven,67/3.2,"Au coin de Kalkoven et Gildehof, à quelques pa...",0,1,1,2,-1,70,1,2020,2,-1,0,0,1,0,0,0,1,4,3,0,0,1,0,0,4.223983,50.932850
2283,9015948,160000,hainaut,7860,Chemin du Comte d'Egmont,11,"comprenant un hall d'entrée, 1 espace séjour l...",0,1,1,1,1,48,1,2017,2,1,0,1,1,0,0,1,0,2,1,0,0,0,0,1,3.830113,50.712304
2060,8979886,192500,hainaut,7060,,,Idéalement situé en pleine campagne et 10 minu...,0,1,0,2,-1,82,1,2017,2,-1,0,0,1,0,0,0,1,2,1,0,0,0,0,1,4.073297,50.577531
1997,9018947,420000,hainaut,7700,,,"FAIRE OFFRE A PARTIR DE 420 000€, sous réserve...",0,4,1,5,-1,315,1,2017,4,-1,0,4,3,2,1,0,0,2,1,0,1,0,0,0,3.213909,50.743335
